In [1]:
import pandas as pd
import gzip
import time
# Install a few python packages using pip
from common import utils
utils.require_package('nltk')
utils.require_package("wget")      # for fetching dataset
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Standard python helper libraries.
from __future__ import print_function
from __future__ import division
import os, sys, time
import collections
import itertools

# Numerical manipulation libraries.
import numpy as np
from scipy import stats, optimize

import nltk
nltk.download('punkt')
from nltk import word_tokenize

# Helper libraries
from common import utils, vocabulary

[nltk_data] Downloading package punkt to /home/arunima/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
#Function to read the amazon review data files
def parse(path):
  print('start parse')
  start_parse = time.time()
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)
  end_parse = time.time()
  print('end parse with time for parse',end_parse - start_parse)

def getDF(path):
  print('start getDF')
  start = time.time()
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  print('end getDF')
  end = time.time()
  print('time taken to load data = ',end-start)
  return pd.DataFrame.from_dict(df, orient='index')
#df = getDF('reviews_Toys_and_Games.json.gz') #old def function corresponding to the step bt step vectorization

In [4]:
df_vid = getDF('reviews_Video_Games.json.gz')
df_toys = getDF('reviews_Toys_and_Games.json.gz')

start getDF
start parse
end parse with time for parse 74.79690313339233
end getDF
time taken to load data =  74.79718041419983
start getDF
start parse
end parse with time for parse 112.6529746055603
end getDF
time taken to load data =  112.65360379219055


In [5]:
df_aut = getDF('reviews_Automotive.json.gz')

start getDF
start parse
end parse with time for parse 65.3999924659729
end getDF
time taken to load data =  65.40044379234314


In [6]:
df_hnk = getDF('reviews_Home_and_Kitchen.json.gz')

start getDF
start parse
end parse with time for parse 220.23961353302002
end getDF
time taken to load data =  220.2400245666504


In [7]:
#Looking at a few exampls of the data.
print('\n Toys reviews summary')
print(df_toys.shape)
print(df_toys.columns)
print(df_toys.head(3))
print('\n Video games reviews summary')
print(df_vid.shape)
print(df_vid.columns)
print(df_vid.head(3))
print('\n Auto reviews summary')
print(df_aut.shape)
print(df_aut.columns)
print(df_aut.head(3))
print('\n Home and Kitchen reviews summary')
print(df_hnk.shape)
print(df_hnk.columns)
print(df_hnk.head(3))


 Toys reviews summary
(2252771, 9)
Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime'],
      dtype='object')
       reviewerID        asin   reviewerName helpful  \
0   AMEVO2LY6VEJA  0000191639  Nicole Soeder  [0, 0]   
1  A3C9CSW3TJITGT  0005069491          Renee  [0, 0]   
2  A31POTIYCKSZ9G  0076561046  So CA Teacher  [0, 0]   

                                          reviewText  overall  \
0  Great product, thank you! Our son loved the pu...      5.0   
1  I love these felt nursery rhyme characters and...      4.0   
2  I see no directions for its use. Therefore I h...      3.0   

                                      summary  unixReviewTime   reviewTime  
0                                     Puzzles      1388016000  12 26, 2013  
1  Charming characters but busy work required      1377561600  08 27, 2013  
2                    No directions for use...      1404864000   07 9, 2014  

 Video games re

In [8]:
#Count by ratings to determine skew in sample.
print('Ratings distribution for toys',df_toys.groupby('overall').count())
print('\n Ratings distribution for video games',df_vid.groupby('overall').count())
print('\n Ratings distribution for automobiles',df_aut.groupby('overall').count())
print('\n Ratings distribution for home and kitchen',df_hnk.groupby('overall').count())

Ratings distribution for toys          reviewerID     asin  reviewerName  helpful  reviewText  summary  \
overall                                                                    
1.0          192993   192993        192435   192993      192993   192993   
2.0          115801   115801        115416   115801      115801   115801   
3.0          193941   193941        193195   193941      193941   193941   
4.0          407884   407884        406255   407884      407884   407884   
5.0         1342152  1342152       1333623  1342152     1342152  1342152   

         unixReviewTime  reviewTime  
overall                              
1.0              192993      192993  
2.0              115801      115801  
3.0              193941      193941  
4.0              407884      407884  
5.0             1342152     1342152  

 Ratings distribution for video games          reviewerID    asin  reviewerName  helpful  reviewText  summary  \
overall                                                  

In [9]:
#Looking at a few examples of review text
print('Toys reviews examples\n')
for i in range(3):
    print(df_toys['reviewerID'].iloc[i])
    print(df_toys['reviewText'].iloc[i])

print('\n Video games reviews examples\n')
for i in range(3):
    print(df_vid['reviewerID'].iloc[i])
    print(df_vid['reviewText'].iloc[i])
    
print('\n Automobile reviews examples\n')
for i in range(3):
    print(df_aut['reviewerID'].iloc[i])
    print(df_aut['reviewText'].iloc[i])
    
print('\n Home and Kitchen reviews examples\n')
for i in range(3):
    print(df_hnk['reviewerID'].iloc[i])
    print(df_hnk['reviewText'].iloc[i])

Toys reviews examples

AMEVO2LY6VEJA
Great product, thank you! Our son loved the puzzles.  They have large pieces yet they are still challenging for a 4 year old.
A3C9CSW3TJITGT
I love these felt nursery rhyme characters and scenes.  The quality of the felt is good, and the illustrations are detailed and pretty.  As noted, the figures and scenes are printed on 2 large sheets of flannel and each individual item needs to be cut out.  This process took me 2 hours of tiny cutting.  To me it does not lend itself to a book form but rather laying out the scenes separately or for use on a flannel board.  However, I love the quiet play it offers for my toddler, and as a former Kindergarten teacher, I understand the value of learning rhyme and its connection to future reading.  Overall, delightful product with some work involved.
A31POTIYCKSZ9G
I see no directions for its use. Therefore I have to make up the games, unfortunately.

 Video games reviews examples

AB9S9279OZ3QO
I haven't gotten aro

In [10]:
#Get the count by unique product id, and % of products and reviews left if we limit to products with at least X reviews
def product_skew(df):
    tempcnt = df.groupby('asin').size().reset_index()
    cnt_total = tempcnt.count()[1]
    sum_total = tempcnt.iloc[:,1].sum()
    cnt_5 = tempcnt[tempcnt.iloc[:,1] > 5].count()[1]
    sum_5 = tempcnt[tempcnt.iloc[:,1] > 5].sum()[1]
    cnt_20 = tempcnt[tempcnt.iloc[:,1] > 20].count()[1]
    sum_20 = tempcnt[tempcnt.iloc[:,1] > 20].sum()[1]
    print('Total','\n\tCount of unique products:',cnt_total,'\n\tSum of their reviews',sum_total)
    print('Total with at least 5 reviews','\n\tCount of unique products:',cnt_5,'Percentage of total {0:.0f}%'.format(cnt_5*100/cnt_total))
    print('\tSum of their reviews',sum_5,'Percentage of total {0:.0f}%'.format(sum_5*100/sum_total))
    print('Total with at least 20 reviews','\n\tCount of unique products:',cnt_20,'Percentage of total {0:.0f}%'.format(cnt_20*100/cnt_total))
    print('\tSum of their reviews',sum_20,'Percentage of total {0:.0f}%'.format(sum_20*100/sum_total))
    return

print('Additional Summary metrics for Toy reviews')
product_skew(df_toys)

print('\nAdditional Summary metrics for Video Games reviews')
product_skew(df_vid)

print('\nAdditional Summary metrics for Auto reviews')
product_skew(df_aut)

print('\nAdditional Summary metrics for Home and Kitchen reviews')
product_skew(df_hnk)


Additional Summary metrics for Toy reviews
Total 
	Count of unique products: 327698 
	Sum of their reviews 2252771
Total with at least 5 reviews 
	Count of unique products: 68782 Percentage of total 21%
	Sum of their reviews 1775109 Percentage of total 79%
Total with at least 20 reviews 
	Count of unique products: 19992 Percentage of total 6%
	Sum of their reviews 1275698 Percentage of total 57%

Additional Summary metrics for Video Games reviews
Total 
	Count of unique products: 50210 
	Sum of their reviews 1324753
Total with at least 5 reviews 
	Count of unique products: 23866 Percentage of total 48%
	Sum of their reviews 1266698 Percentage of total 96%
Total with at least 20 reviews 
	Count of unique products: 10904 Percentage of total 22%
	Sum of their reviews 1124236 Percentage of total 85%

Additional Summary metrics for Auto reviews
Total 
	Count of unique products: 320112 
	Sum of their reviews 1373768
Total with at least 5 reviews 
	Count of unique products: 42052 Percentage o

In [11]:
#Create train,dev,test split
from sklearn.model_selection import train_test_split
train_toys,devtest = train_test_split(df_toys, test_size=0.4)
dev_toys,test_toys = train_test_split(devtest,test_size = 0.5)
print('Toy reviews train, dev and test set dataframe shape:',train_toys.shape,dev_toys.shape,test_toys.shape)

#For Video games reviews
train_vid,devtest = train_test_split(df_vid, test_size=0.4)
dev_vid,test_vid = train_test_split(devtest,test_size = 0.5)
print('Video games reviews train, dev and test set dataframe shape:',train_vid.shape,dev_vid.shape,test_vid.shape)

#For Auto reviews
train_aut,devtest = train_test_split(df_aut, test_size=0.4)
dev_aut,test_aut = train_test_split(devtest,test_size = 0.5)
print('Auto reviews train, dev and test set dataframe shape:',train_aut.shape,dev_aut.shape,test_aut.shape)

#For Home and Kitchen reviews
train_hnk,devtest = train_test_split(df_hnk, test_size=0.4)
dev_hnk,test_hnk = train_test_split(devtest,test_size = 0.5)
print('Home and Kitchen reviews train, dev and test set dataframe shape:',train_hnk.shape,dev_hnk.shape,test_hnk.shape)

Toy reviews train, dev and test set dataframe shape: (1351662, 9) (450554, 9) (450555, 9)
Video games reviews train, dev and test set dataframe shape: (794851, 9) (264951, 9) (264951, 9)
Auto reviews train, dev and test set dataframe shape: (824260, 9) (274754, 9) (274754, 9)
Home and Kitchen reviews train, dev and test set dataframe shape: (2552355, 9) (850785, 9) (850786, 9)


In [31]:
#Function to create a smaller sized train and dev data set. Enables testing accuracy for different sizes.
#Also binarizes the labels. Ratings of 1,2 and to 0; Ratings of 4,5 to 1.

def set_df_size(size,data_train,data_dev):
    size_train = size
    len_max_train = data_train[data_train.overall!=3].shape[0] #max possible length of train data set taking out the 3 ratings.
    #print("Number of reviews with ratings != 3 in train set",len_max_train)
    temp_size_train = min(len_max_train,size_train)

    len_max_dev = data_dev[data_dev.overall!=3].shape[0]
    #print("Number of reviews with ratings != 3 in dev set",len_max_dev)
    temp_size_dev = min(len_max_dev,int(0.3*temp_size_train)) #making the dev set about 0.3 times the train set.

    temp_train_data = data_train[data_train.overall != 3][:temp_size_train]
    #print('Size of train data',temp_train_data.shape)
    #print(temp_train_data.groupby('overall').count())
    #print(temp_train_toys[:5])

    temp_dev_data = data_dev[data_dev.overall!=3][:temp_size_dev]
    #print('Size of dev data',temp_dev_data.shape)
    #print(temp_dev_data.groupby('overall').count())
    #print(temp_dev_data[:2])
    
    #Binarize ratings
    temp_train_y = np.zeros(temp_size_train)
    temp_train_y[temp_train_data.overall > 3] = 1
    temp_dev_y = np.zeros(temp_size_dev)
    temp_dev_y[temp_dev_data.overall>3] = 1
    #print('binarized y shape',temp_train_y.shape,temp_dev_y.shape)
    #print(temp_dev_y[:20],data_dev.overall[:20])
    return temp_train_data,temp_dev_data,temp_train_y,temp_dev_y

In [33]:
list_df = ['toys','vid','aut','hnk'] #list of keys that refer to each dataframe. Adding a new dataframe would require updating this list
dict_train_df = {} #Dict to store train input data frame for each domain, can be accessed by using domain name as key
dict_dev_df = {} #Dict to store dev input data frame for each domain, can be accessed by using domain name as key
dict_train_y = {} #Dict to store binarized train data label for each domain
dict_dev_y = {} #Dict to store binarized dev data label for each domain
#print(len(dict_train_df))

def create_sized_data(size = 100000):
    size_train = size #Set size of train set here. This is a hyperparameter.
    key = list_df[0]
    #print('Toys reviews\n')
    dict_train_df[key], dict_dev_df[key], dict_train_y[key], dict_dev_y[key] = set_df_size(size_train,train_toys,dev_toys)
    #print('\n Video games reviews\n')
    key = list_df[1]
    dict_train_df[key], dict_dev_df[key], dict_train_y[key], dict_dev_y[key] = set_df_size(size_train,train_vid,dev_vid)
    #print('\n Auto reviews\n')
    key = list_df[2]
    dict_train_df[key], dict_dev_df[key], dict_train_y[key], dict_dev_y[key] = set_df_size(size_train,train_aut,dev_aut)
    #print('\n Home and Kitchen reviews\n')
    key = list_df[3]
    dict_train_df[key], dict_dev_df[key], dict_train_y[key], dict_dev_y[key] = set_df_size(size_train,train_hnk,dev_hnk)
    
create_sized_data()
#print(len(dict_train_df))

0


In [95]:
#Converting reviews to sparse matrix of word ids with count vectorizer, and using Naive Bayes to make the prediction.
#This section also creates the count_vectorizer and Naive Bayes models for each domain to be used to test transfer learning

from sklearn.naive_bayes import MultinomialNB
dict_vectorizers = {} #Dict to store the count_vectorizer model developed on each domain
dict_train_ids = {} #Dict to store train data reviews as sparse matrix of word ids
dict_dev_ids = {} #Dict to store dev data reviews as sparse matrix of word ids
dict_nb = {} #Dict to store naive bayes model developed on each domain. Assumes input features are developed using the corresponding count_vectorizer
dict_dev_ypred = {} #Dict to store dev predictions

def create_base_NB_models():
    for key in list_df:

        #Converting ratings to tokenized word id counts as a sparse matrix using count_vectorizer
        dict_vectorizers[key] = CountVectorizer()
        dict_train_ids[key] = dict_vectorizers[key].fit_transform(dict_train_df[key].reviewText)
        dict_dev_ids[key] = dict_vectorizers[key].transform(dict_dev_df[key].reviewText)
        print("Number words in training corpus for",key,len(dict_vectorizers[key].get_feature_names()))
        #print(key,'dataset id shapes',dict_train_ids[key].shape, dict_dev_ids[key].shape)

        #Building a Naive Bayes model to predict the ratings
        dict_nb[key] = MultinomialNB()
        dict_nb[key].fit(dict_train_ids[key],dict_train_y[key])
        dict_dev_ypred[key] = dict_nb[key].predict(dict_dev_ids[key])
        acc = accuracy_score(dict_dev_y[key], dict_dev_ypred[key])
        print("Accuracy on",key,"dev set for binary prediction with toys naive bayes model: {:.02%}".format(acc))

def print_base_NB_details():
    for key in list_df:
      print('Classification report for',key,'\n',classification_report(dict_dev_y[key], dict_dev_ypred[key]))  
        
create_base_NB_models()
print_base_NB_details()

Number words in training corpus for toys 64698
Accuracy on toys dev set for binary prediction with toys naive bayes model: 92.17%
Number words in training corpus for vid 98625
Accuracy on vid dev set for binary prediction with toys naive bayes model: 89.22%
Number words in training corpus for aut 59179
Accuracy on aut dev set for binary prediction with toys naive bayes model: 91.56%
Number words in training corpus for hnk 57706
Accuracy on hnk dev set for binary prediction with toys naive bayes model: 91.56%
Classification report for toys 
              precision    recall  f1-score   support

        0.0       0.73      0.74      0.74      4476
        1.0       0.96      0.95      0.95     25524

avg / total       0.92      0.92      0.92     30000

Classification report for vid 
              precision    recall  f1-score   support

        0.0       0.72      0.70      0.71      5723
        1.0       0.93      0.94      0.93     24277

avg / total       0.89      0.89      0.89   

In [99]:
#Accuracy of transfer learning

dict_transfer_ids = {} #Dictionary to store the dev vector ids for dataframe A(df) using the count_vectorizer of dataframe B(vect)
transfer_results = pd.DataFrame(index=list_df,columns=list_df) #Dataframe to store accuracy on transfer. Col = Model, row = dataframe

def estimate_transfer_accuracy():
    for vectKey in list_df:
        dict_transfer_ids[vectKey] = {}
        #print('vectKey',vectKey)
        for dfKey in list_df:
            #print('dfKey',dfKey)
            dict_transfer_ids[vectKey][dfKey] = dict_vectorizers[vectKey].transform(dict_dev_df[dfKey].reviewText)
            #print(dfKey,'dataset using ',vectKey,' count vectorizer, id shapes',dict_transfer_ids[vectKey][dfKey].shape)
            dict_dev_ypred = dict_nb[vectKey].predict(dict_transfer_ids[vectKey][dfKey])
            acc = accuracy_score(dict_dev_y[dfKey], dict_dev_ypred)
            #print("Accuracy on ",dfKey," dev set for binary prediction with ", vectKey," naive bayes model: {:.02%}".format(acc))
            transfer_results[vectKey][dfKey] = acc

    print("Effectiveness of transfer learning with Naive Bayes:")
    print("Accuracy of rating predictions")
    print("Colums = source domain, Rows = target domain\n")
    print(transfer_results.to_string(float_format = '{:.01%}'.format))

estimate_transfer_accuracy()

Effectiveness of transfer learning with Naive Bayes:
Accuracy of rating predictions
Colums = source domain, Rows = target domain

      toys   vid   aut   hnk
toys 92.2% 91.3% 91.3% 91.1%
vid  86.6% 89.2% 87.4% 87.3%
aut  75.4% 78.8% 91.6% 83.6%
hnk  85.8% 86.5% 90.9% 91.6%


In [97]:
#Calculating and displaying as transfer loss
transfer_loss = pd.DataFrame(index=list_df,columns=list_df) #Dataframe to store loss in accuracy on transfer. Col = Model, row = dataframe
def estimate_transfer_loss():
    for A in list_df:
        for B in list_df:
            transfer_loss[A][B] = transfer_results[B][B] - transfer_results[A][B]
    print("Transfer loss on rating predictions")
    print("Colums = source domain, Rows = target domain\n")
    print(transfer_loss.to_string(float_format = '{:.01%}'.format))

estimate_transfer_loss()

Transfer loss on rating predictions
Colums = source domain, Rows = target domain

      toys   vid   aut  hnk
toys  0.0%  0.4% -0.1% 0.3%
vid   2.9%  0.0%  0.9% 1.4%
aut  19.8% 15.3%  0.0% 9.4%
hnk   8.1%  6.0%  0.2% 0.0%


In [73]:
#for size in (50000,100000,250000,500000,1000000):
for size in (5000,100000):
    print("\n Train data_set size =",size)
    create_sized_data(size = size)
    create_base_NB_models()
    estimate_transfer_accuracy()
    estimate_transfer_loss()


 Train data_set size = 5000
Number words in training corpus for toys 7133
Accuracy on toys dev set for binary prediction with toys naive bayes model: 90.53%
Number words in training corpus for vid 10570
Accuracy on vid dev set for binary prediction with toys naive bayes model: 88.87%
Number words in training corpus for aut 6610
Accuracy on aut dev set for binary prediction with toys naive bayes model: 91.40%
Number words in training corpus for hnk 7255
Accuracy on hnk dev set for binary prediction with toys naive bayes model: 89.80%
Effectiveness of transfer learning with Naive Bayes:
Accuracy of rating predictions
Colums = source domain, Rows = target domain

      toys   vid   aut   hnk
toys 90.5% 88.1% 89.9% 90.0%
vid  86.1% 88.9% 86.1% 86.4%
aut  78.5% 82.5% 91.4% 88.8%
hnk  86.0% 87.4% 88.5% 89.8%
Transfer loss on rating predictions
Colums = source domain, Rows = target domain

      toys  vid  aut  hnk
toys  0.0% 2.5% 0.7% 0.5%
vid   2.7% 0.0% 2.8% 2.5%
aut  12.9% 8.9% 0.0% 2.6%

In [63]:
from scipy.stats import entropy
#from scipy.sparse.linalg import norm
from numpy.linalg import norm

def JSD(P, Q):
    _P = P / norm(P, ord=1)
    _Q = Q / norm(Q, ord=1)
    _M = 0.5 * (_P + _Q)
    return 0.5 * (entropy(_P, _M) + entropy(_Q, _M))

In [104]:
#print(dict_train_ids['toys'])
JSD_results = pd.DataFrame(index=list_df,columns=list_df)

for key1 in list_df:
    for key2 in list_df:
        dict_train_ids_1 = dict_train_ids[key1].sum(axis=0).T
        dict_train_ids_2 = dict_transfer_ids[key1][key2].sum(axis=0).T
        #print(dict_train_ids[key1].shape,dict_train_ids_1.shape,dict_train_ids_2.shape)
        JSD_results[key1][key2] = JSD(dict_train_ids_1,dict_train_ids_2)
        
print('JS Divergence')
print(JSD_results)

#Need to check calculations

JS Divergence
                    toys                 vid                 aut  \
toys  [0.00615937165999]   [0.0698327292496]   [0.0775560004658]   
vid    [0.0665743115835]  [0.00539721105095]   [0.0981619830986]   
aut    [0.0791695973189]    [0.103475630761]  [0.00598173625196]   
hnk     [0.065517513502]    [0.101538756016]   [0.0585633518635]   

                     hnk  
toys     [0.06459895224]  
vid    [0.0977522244749]  
aut    [0.0595313701389]  
hnk   [0.00497069111742]  


### Keeping track of results from test runs
With number in train set = 10000 (excl 3 ratings)    
    Accuracy on dev set for binary prediction: 88.74%
    Accuracy on dev set for 4 level (1,2,4,5) prediction: 67.16%
    Vocab Size : 38696
    
With number in train set = 50000 (excl 3 ratings)   
    Accuracy on dev set for binary prediction: 91.33%   
    Accuracy on dev set for 4 level (1,2,4,5) prediction: 69.33% 
    Vocab Size : ~ ..
    
With number in train set = 100000 (excl 3 ratings)
    Accuracy on dev set for binary prediction: 91.56%   
    Accuracy on dev set for 4 level (1,2,4,5) prediction: 70.42%
    Vocab Size : 105304

With number in train set = 500000, dev set = 150000 (excl 3 ratings)    
    Accuracy on dev set for binary prediction: 91.73%
    Accuracy on dev set for 4 level (1,2,4,5) prediction: 70.95%
    vocab size 307822
    
With number in train set = 1200000, dev set = 360000 (excl 3 ratings)    
    Accuracy on dev set for binary prediction: 91.92%
    Accuracy on dev set for 4 level (1,2,4,5) prediction: 71.24%
    vocab size 674074 (not repeated with correction for vocab)
    
### Output from trying different pre-processing with the toys review set.
 
 Accuracy on dev set for binary prediction: 91.69%
classification report naive bayes binary classification 
              precision    recall  f1-score   support

        0.0       0.70      0.77      0.74     22472
        1.0       0.96      0.94      0.95    127528

avg / total       0.92      0.92      0.92    150000

Accuracy on dev set for binary prediction with count vectorizer: 91.92%
classification report naive bayes binary classification with count vectorizer 
              precision    recall  f1-score   support

        0.0       0.71      0.79      0.75     22472
        1.0       0.96      0.94      0.95    127528

avg / total       0.92      0.92      0.92    150000

Accuracy on dev set for binary prediction with tfidf: 90.13%
classification report naive bayes binary classification with tfidf 
              precision    recall  f1-score   support

        0.0       0.90      0.38      0.54     22472
        1.0       0.90      0.99      0.94    127528

avg / total       0.90      0.90      0.88    150000

Accuracy on dev set for 4 level (1,2,4,5) prediction: 70.91%
classification report naive bayes multinomial classification with tfidf 
              precision    recall  f1-score   support

          1       0.60      0.74      0.66     13975
          2       0.32      0.05      0.09      8497
          4       0.42      0.34      0.37     29733
          5       0.80      0.87      0.83     97795

avg / total       0.68      0.71      0.68    150000

### Output from simple ratings prediction with video games review set.

train set size : 10000, dev set size : 3000
Accuracy on dev set for binary prediction with count vectorizer: 88.93%
classification report naive bayes binary classification with count vectorizer 
              precision    recall  f1-score   support

        0.0       0.77      0.54      0.64       534
        1.0       0.91      0.96      0.93      2466

avg / total       0.88      0.89      0.88      3000

Accuracy on dev set for binary prediction with tfidf: 84.93%
classification report naive bayes binary classification with tfidf 
              precision    recall  f1-score   support

        0.0       0.95      0.16      0.28       534
        1.0       0.85      1.00      0.92      2466

avg / total       0.86      0.85      0.80      3000

Using SVM, with Count Vectorizer pre-processing:
Accuracy on dev set for binary prediction: 82.20%
classification report svm              precision    recall  f1-score   support

        0.0       0.00      0.00      0.00       534
        1.0       0.82      1.00      0.90      2466

avg / total       0.68      0.82      0.74      3000

time taken for SVM 48.42102265357971

Using SVM with TFIDF pre-processing:
Accuracy on dev set for binary prediction: 82.20%
classification report svm              precision    recall  f1-score   support

        0.0       0.00      0.00      0.00       534
        1.0       0.82      1.00      0.90      2466

avg / total       0.68      0.82      0.74      3000

train set size : 100000, dev set size : 30000
Accuracy on dev set for binary prediction with count vectorizer: 89.12%
classification report naive bayes binary classification with count vectorizer 
              precision    recall  f1-score   support

        0.0       0.72      0.71      0.71      5728
        1.0       0.93      0.93      0.93     24272

avg / total       0.89      0.89      0.89     30000

Accuracy on dev set for binary prediction with tfidf: 86.04%
classification report naive bayes binary classification with tfidf 
              precision    recall  f1-score   support

        0.0       0.91      0.30      0.45      5728
        1.0       0.86      0.99      0.92     24272

avg / total       0.87      0.86      0.83     30000



### Results for transfer learning from toys to video games
number words in training corpus for toys: 63984
toys dataset id shapes (100000, 63984) (30000, 63984)
number words in training corpus for video games: 98899
videos dataset id shapes (100000, 98899) (30000, 98899)
number words in training corpus for automobiles: 59468
automobile dataset id shapes (100000, 59468) (30000, 59468)
number words in training corpus for home and kitchen: 57884
home and kitchen dataset id shapes (100000, 57884) (30000, 57884)

Accuracy on toys dev set for binary prediction with toys naive bayes model: 92.23%   
Corresponding classification report              precision    recall  f1-score   support

        0.0       0.74      0.74      0.74      4503
        1.0       0.95      0.95      0.95     25497

avg / total       0.92      0.92      0.92     30000

Accuracy on video games dev set for binary prediction with video games naive bayes model: 89.16%   
Corresponding classification report              precision    recall  f1-score   support

        0.0       0.72      0.71      0.71      5725
        1.0       0.93      0.93      0.93     24275

avg / total       0.89      0.89      0.89     30000

Accuracy on autos dev set for binary prediction with autos naive bayes model: 91.93%   
Corresponding classification report              precision    recall  f1-score   support

        0.0       0.78      0.61      0.69      4323
        1.0       0.94      0.97      0.95     25677

avg / total       0.91      0.92      0.92     30000

Accuracy on home and kitchen dev set for binary prediction with home and kitchen naive bayes model: 91.37%   
Corresponding classification report              precision    recall  f1-score   support

        0.0       0.76      0.71      0.73      5072
        1.0       0.94      0.96      0.95     24928

avg / total       0.91      0.91      0.91     30000

### Transfer learning:

Accuracy on video games dev set for binary prediction with toys naive bayes model: 86.99%   
Corresponding classification report              precision    recall  f1-score   support

        0.0       0.66      0.65      0.66      5725
        1.0       0.92      0.92      0.92     24275

avg / total       0.87      0.87      0.87     30000

Accuracy on automobiles dev set for binary prediction with toys naive bayes model: 76.06%   
Corresponding classification report              precision    recall  f1-score   support

        0.0       0.36      0.88      0.51      4323
        1.0       0.97      0.74      0.84     25677

avg / total       0.88      0.76      0.79     30000

Accuracy on home and kitchen dev set for binary prediction with toys naive bayes model: 85.78%   
Corresponding classification report              precision    recall  f1-score   support

        0.0       0.55      0.85      0.67      5072
        1.0       0.97      0.86      0.91     24928

avg / total       0.90      0.86      0.87     30000

Accuracy on toys dev set for binary prediction with video games naive bayes model: 91.53%   
Corresponding classification report              precision    recall  f1-score   support

        0.0       0.76      0.63      0.69      4503
        1.0       0.94      0.97      0.95     25497

avg / total       0.91      0.92      0.91     30000

Accuracy on automobiles dev set for binary prediction with video games naive bayes model: 80.50%   
Corresponding classification report              precision    recall  f1-score   support

        0.0       0.41      0.77      0.53      4323
        1.0       0.96      0.81      0.88     25677

avg / total       0.88      0.81      0.83     30000

### Results from running Naive Bayes for all 4 dfs, with different train set size.

Train data_set size = 50000
Number words in training corpus for toys 45973
Accuracy on toys dev set for binary prediction with toys naive bayes model: 92.46%
Number words in training corpus for vid 68303
Accuracy on vid dev set for binary prediction with toys naive bayes model: 89.37%
Number words in training corpus for aut 41130
Accuracy on aut dev set for binary prediction with toys naive bayes model: 91.57%
Number words in training corpus for hnk 41378
Accuracy on hnk dev set for binary prediction with toys naive bayes model: 91.43%
Effectiveness of transfer learning with Naive Bayes:
Accuracy of rating predictions
Colums = source domain, Rows = target domain

      toys   vid   aut   hnk
toys 92.5% 91.4% 90.9% 91.2%
vid  87.1% 89.4% 86.8% 87.3%
aut  76.7% 80.0% 91.6% 84.6%
hnk  86.1% 86.5% 90.3% 91.4%
Transfer loss on rating predictions
Colums = source domain, Rows = target domain

      toys   vid  aut  hnk
toys  0.0%  1.0% 1.6% 1.3%
vid   2.2%  0.0% 2.5% 2.1%
aut  14.9% 11.5% 0.0% 7.0%
hnk   5.3%  4.9% 1.1% 0.0%

 Train data_set size = 100000
Number words in training corpus for toys 64698
Accuracy on toys dev set for binary prediction with toys naive bayes model: 92.17%
Number words in training corpus for vid 98625
Accuracy on vid dev set for binary prediction with toys naive bayes model: 89.22%
Number words in training corpus for aut 59179
Accuracy on aut dev set for binary prediction with toys naive bayes model: 91.56%
Number words in training corpus for hnk 57706
Accuracy on hnk dev set for binary prediction with toys naive bayes model: 91.56%
Effectiveness of transfer learning with Naive Bayes:
Accuracy of rating predictions
Colums = source domain, Rows = target domain

      toys   vid   aut   hnk
toys 92.2% 91.3% 91.3% 91.1%
vid  86.6% 89.2% 87.4% 87.3%
aut  75.4% 78.8% 91.6% 83.6%
hnk  85.8% 86.5% 90.9% 91.6%
Transfer loss on rating predictions
Colums = source domain, Rows = target domain

      toys   vid  aut  hnk
toys  0.0%  0.8% 0.9% 1.0%
vid   2.6%  0.0% 1.8% 1.9%
aut  16.2% 12.8% 0.0% 8.0%
hnk   5.8%  5.1% 0.7% 0.0%

 Train data_set size = 250000
Number words in training corpus for toys 104512
Accuracy on toys dev set for binary prediction with toys naive bayes model: 91.92%
Number words in training corpus for vid 165841
Accuracy on vid dev set for binary prediction with toys naive bayes model: 89.01%
Number words in training corpus for aut 97914
Accuracy on aut dev set for binary prediction with toys naive bayes model: 91.87%
Number words in training corpus for hnk 93561
Accuracy on hnk dev set for binary prediction with toys naive bayes model: 91.58%
Effectiveness of transfer learning with Naive Bayes:
Accuracy of rating predictions
Colums = source domain, Rows = target domain

      toys   vid   aut   hnk
toys 91.9% 91.4% 91.8% 91.3%
vid  86.1% 89.0% 88.0% 87.5%
aut  72.7% 78.0% 91.9% 83.6%
hnk  84.9% 86.3% 91.5% 91.6%
Transfer loss on rating predictions
Colums = source domain, Rows = target domain

      toys   vid  aut  hnk
toys  0.0%  0.6% 0.1% 0.7%
vid   2.9%  0.0% 1.0% 1.6%
aut  19.2% 13.8% 0.0% 8.3%
hnk   6.7%  5.3% 0.1% 0.0%

 Train data_set size = 500000
Number words in training corpus for toys 151534
Accuracy on toys dev set for binary prediction with toys naive bayes model: 91.81%
Number words in training corpus for vid 249256
Accuracy on vid dev set for binary prediction with toys naive bayes model: 88.99%
Number words in training corpus for aut 144802
Accuracy on aut dev set for binary prediction with toys naive bayes model: 92.06%
Number words in training corpus for hnk 137575
Accuracy on hnk dev set for binary prediction with toys naive bayes model: 91.55%
Effectiveness of transfer learning with Naive Bayes:
Accuracy of rating predictions
Colums = source domain, Rows = target domain

      toys   vid   aut   hnk
toys 91.8% 91.4% 91.9% 91.4%
vid  85.9% 89.0% 88.4% 87.9%
aut  72.8% 78.7% 92.1% 83.6%
hnk  84.5% 86.9% 91.7% 91.6%
Transfer loss on rating predictions
Colums = source domain, Rows = target domain

      toys   vid   aut  hnk
toys  0.0%  0.4% -0.1% 0.5%
vid   3.1%  0.0%  0.6% 1.1%
aut  19.3% 13.3%  0.0% 8.5%
hnk   7.1%  4.7% -0.2% 0.0%

 Train data_set size = 1000000
Number words in training corpus for toys 224573
Accuracy on toys dev set for binary prediction with toys naive bayes model: 91.74%
Number words in training corpus for vid 309416
Accuracy on vid dev set for binary prediction with toys naive bayes model: 88.89%
Number words in training corpus for aut 185933
Accuracy on aut dev set for binary prediction with toys naive bayes model: 92.03%
Number words in training corpus for hnk 204991
Accuracy on hnk dev set for binary prediction with toys naive bayes model: 91.50%
Effectiveness of transfer learning with Naive Bayes:
Accuracy of rating predictions
Colums = source domain, Rows = target domain

      toys   vid   aut   hnk
toys 91.7% 91.5% 91.9% 91.4%
vid  85.8% 88.9% 88.7% 88.0%
aut  73.0% 78.6% 92.0% 83.3%
hnk  84.4% 86.5% 91.8% 91.5%
Transfer loss on rating predictions
Colums = source domain, Rows = target domain

      toys   vid   aut  hnk
toys  0.0%  0.3% -0.2% 0.3%
vid   3.1%  0.0%  0.2% 0.9%
aut  19.0% 13.5%  0.0% 8.7%
hnk   7.1%  5.0% -0.3% 0.0%

from scipy.stats import entropy
